<a href="https://colab.research.google.com/github/dadaocao/ai-music-generator/blob/main/AI_Lyrics_Generator_Team_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# See integration and demo on: https://github.com/dadaocao/ai-music-generator.git

# Install
!pip install transformers datasets torch kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"melodyxy","key":"8f36a50cf3111883efcf083aa058fc6e"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d karnikakapoor/lyrics
!unzip lyrics.zip

Dataset URL: https://www.kaggle.com/datasets/karnikakapoor/lyrics
License(s): CC0-1.0
Archive:  lyrics.zip
  inflating: Songs.csv               


In [ ]:
import pandas as pd
from datasets import Dataset

# Loading data
df = pd.read_csv("Songs.csv")

# Check the structure
print(df.head())

# Clean data: Delete empty values ​​+ merge Artist and Title as context
df = df.dropna(subset=["Lyrics"])  # Delete lines without lyrics
df["context"] = "Artist: " + df["Artist"] + " | Title: " + df["Title"]  # Added context column
df = df[["context", "Lyrics"]]  # Keep only the required columns
print(df.head())

         Artist                     Title  \
0  Taylor Swift                  cardigan   
1  Taylor Swift                     exile   
2  Taylor Swift                     Lover   
3  Taylor Swift                     the 1   
4  Taylor Swift  Look What You Made Me Do   

                                              Lyrics  
0  Vintage tee, brand new phone\nHigh heels on co...  
1  I can see you standing, honey\nWith his arms a...  
2  We could leave the Christmas lights up 'til Ja...  
3  I'm doing good, I'm on some new shit\nBeen say...  
4  I don't like your little games\nDon't like you...  
                                             context  \
0             Artist: Taylor Swift | Title: cardigan   
1                Artist: Taylor Swift | Title: exile   
2                Artist: Taylor Swift | Title: Lover   
3                Artist: Taylor Swift | Title: the 1   
4  Artist: Taylor Swift | Title: Look What You Ma...   

                                              Lyrics  
0  Vint

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)  # 90% train，10% test

# demo
print(dataset["train"][0])

{'context': 'Artist: Billie Eilish | Title: Billie Bossa Nova', 'Lyrics': "Mm-mm-mm, mm-mm\nNa-na-na\n\nLove when it comes without a warning\n'Cause waiting for it gets so boring\nA lot can change in twenty seconds\nA lot can happen in the dark\nLove when it makes you lose your bearings\nSome information's not for sharing\nUse different names at hotel check-ins\nIt's hard to stop it once it starts (It starts)\n\nI'm not sentimental\nBut there's somethin' 'bout the way you look tonight, mm\nMakes me wanna take a picture\nMake a movie with you that we'd have to hide\n\nYou better lock your phone (Oh)\nAnd look at me whеn you're alone\nWon't take a lot to gеt you goin' (Oh)\nI'm sorry if it's torture though\nI know, I know\n\nIt might be more of an obsession\nYou really make a strong impression (You sure do)\nNobody saw me in the lobby (Saw me in the lobby)\nNobody saw me in your arms, mm\n\nI'm not sentimental\nBut there's somethin' 'bout the way you look tonight, mm ('Bout the way you l

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # set token

def tokenize_function(examples):
    # Merge context and lyrics
    inputs = [f"{ctx}\nLyrics:\n{lyric}" for ctx, lyric in zip(examples["context"], examples["Lyrics"])]
    # Add labels
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"  # Return PyTorch
    )
    tokenized["labels"] = tokenized["input_ids"].clone()  # Copy input_ids as labels
    return tokenized


tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/670 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset["train"].features)
print(tokenized_dataset["train"][0].keys())

{'context': Value(dtype='string', id=None), 'Lyrics': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
dict_keys(['context', 'Lyrics', 'input_ids', 'attention_mask', 'labels'])


In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

# 1. Load model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 2. Training parameter configuration
training_args = TrainingArguments(
    output_dir="./lyrics_finetuned",  # Set path
    per_device_train_batch_size=8,    # Adjustment according to GPU memory
    num_train_epochs=2,               # Epoch
    logging_steps=100,                # Print logs every 100 steps
    save_steps=500,                   # Save the model every 500 steps
    evaluation_strategy="steps",      # Evaluate by step
    eval_steps=500,                   # Evaluate every 500 steps
    fp16=True,                        # GPU acceleration (must support mixed precision)?
    remove_unused_columns=False,      # Prevent labels from being automatically deleted
    report_to="none"                  # Disable W&B logging
)

# 3. Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# 4. Start training!
trainer.train()

# 5. Save the finely tuned model
model.save_pretrained("./my_lyrics_model")
tokenizer.save_pretrained("./my_lyrics_model")  # Save word analyzer

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


('./my_lyrics_model/tokenizer_config.json',
 './my_lyrics_model/special_tokens_map.json',
 './my_lyrics_model/vocab.json',
 './my_lyrics_model/merges.txt',
 './my_lyrics_model/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Loading fine-tuned models and word participlers
model_path = "./my_lyrics_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Loading fine-tuned models and word participlers
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Model loaded on: {device}")

Model loaded on: cpu


In [ ]:
def generate_lyrics(artist, title, max_length=200, temperature=0.7):
    # Prompt input
    prompt = f"Artist: {artist} | Title: {title}\nLyrics:\n"

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate lyrics
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        do_sample=True,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode output
    lyrics = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return lyrics

In [ ]:
# Test Case：Generate Taylor Swift style lyrics
lyrics1 = generate_lyrics(
    artist="Taylor Swift",
    title="Midnight Rain",
    temperature=0.9  # Control creativity/repeatability（0.1~1.0）
)
print(lyrics1)

Artist: Taylor Swift | Title: Midnight Rain
Lyrics:
Mama, I'll be right back
Baby, you never know what's to come
I can't wait until I hear it from you
Slay my face on the ground
It's all mine
I'll be right back

But it's not right, it's not right

It's not right I'm gonna kill you
I can't wait to get back
I can't wait to get back

Oh, I guess that means you're gonna stay
I don't wanna kill you

Mama, it's gonna be fine
I can't wait for you

Mama, no more pain
Oh my god
Baby, I love you

Mama, you're such a lovely girl
Oh, I like you
Oh, I like you

Baby, you're so much more than me
Oh, I just had to have you
Oh


In [ ]:
from google.colab import files
!zip -r my_lyrics_model.zip ./my_lyrics_model
files.download("my_lyrics_model.zip")

  adding: my_lyrics_model/ (stored 0%)
  adding: my_lyrics_model/special_tokens_map.json (deflated 74%)
  adding: my_lyrics_model/tokenizer_config.json (deflated 56%)
  adding: my_lyrics_model/generation_config.json (deflated 24%)
  adding: my_lyrics_model/model.safetensors (deflated 7%)
  adding: my_lyrics_model/config.json (deflated 51%)
  adding: my_lyrics_model/vocab.json (deflated 68%)
  adding: my_lyrics_model/merges.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>